<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to create new OBS IPF file. - THIS IS JUST A BACKUP. DO NOT RUN IT, IT'S SLOW
</span><br>
It was discovered that some OBS are not assigned to the correct layer. That will be corrected via this script.

# 0. Options

In [1]:
import WS_Mdl as WS
import pandas as pd
import os
import numpy as np
from scipy.spatial import cKDTree

In [2]:
import imod

In [3]:
os.getcwd()

'c:\\OD\\WS_Mdl\\code\\PrP\\make_OBS_IPF'

In [4]:
MdlN = 'NBr8'

In [5]:
MdlN_B = 'NBr7'

In [6]:
d_paths = WS.get_MdlN_paths(MdlN)

# 1. Read previous IPF file.
B is NBr5, but NBr7 is the same, but with corrected column names (helps with PP automation etc.).

In [7]:
DF_IPF_B = WS.read_IPF_Spa(f'../../../models/NBr/In/OBS/{MdlN_B}/ijkset_selectie_{MdlN_B}.ipf')

# 2. Read TOP and BOT arrays 

In [8]:
path_TOP = r'C:\OD\WS_Mdl\models\NBr\In\TOP'
l_TOP = [os.path.join(path_TOP, i) for i in os.listdir(path_TOP) if 'idf' in i]
TOP = imod.formats.idf.open(l_TOP, pattern="{name}_L{layer}_")

In [9]:
path_BOT = r'C:\OD\WS_Mdl\models\NBr\In\BOT'
l_BOT = [os.path.join(path_BOT, i) for i in os.listdir(path_BOT) if 'idf' in i]
BOT = imod.formats.idf.open(l_BOT, pattern="{name}_L{layer}_")

In [10]:
type(TOP)

xarray.core.dataarray.DataArray

# 3. Match L based on TOP and BOT values (compared to filter top and bot)

In [11]:
TOP_x, TOP_y = TOP.coords['x'].values, TOP.coords['y'].values # Make sure x and y coordinates exist in TOP as 1D arrays

In [12]:
# Prepare output containers
case1_layers, case2_layers, case3_layers, case4_layers, combined_layers, match_distances, combined_TOP_vals, combined_BOT_vals = ([] for i in range(8))

In [ ]:
for _, row in DF_IPF_B.iterrows(): # Loop through each row in IPF_Mdl
    if _%100==0:
        print(f'{int(_)}/{DF_IPF_B.shape[0]}')
    
    # Find closest grid point in TOP
    dx = TOP_x - row['X']
    dy = TOP_y - row['Y']
    dist_matrix = np.sqrt((dy[:, None])**2 + (dx[None, :])**2)
    iy, ix = np.unravel_index(np.argmin(dist_matrix), dist_matrix.shape)
    min_dist = dist_matrix[iy, ix]

    flt_top = row['filtertoplevel']
    flt_bot = row['filterbottomlevel']
    n_layers = TOP.shape[0]

    # Initialize per-row results
    layer_case1 = None
    layer_case2 = None
    layer_case3 = None
    layers_case4 = []

    # Loop over layers
    for k in range(n_layers):
        top_k = TOP[k, iy, ix]
        bot_k = BOT[k, iy, ix]

        in_top  = bot_k < flt_top < top_k
        in_bot  = bot_k < flt_bot < top_k
        in_full = flt_bot < bot_k and flt_top > top_k

        if in_top:
            layer_case1 = k
        if in_bot:
            layer_case2 = k
        if in_top and in_bot:
            layer_case3 = k
        if in_full:
            layers_case4.append(k)

    # Combine and convert to 1-based indexing
    matched_layers = sorted(set(
        [l for l in [layer_case1, layer_case2, layer_case3] if l is not None] + layers_case4
    ))
    matched_layers_1based = [l + 1 for l in matched_layers]

    # Get combined TOP and BOT if any layer matched
    if matched_layers:
        shallowest_k = min(matched_layers)
        deepest_k    = max(matched_layers)
        top_val = float(TOP[shallowest_k, iy, ix].values)
        bot_val = float(BOT[deepest_k,    iy, ix].values)
    else:
        top_val = None
        bot_val = None

    # Store results (with 1-based indexing)
    case1_layers.append(layer_case1 + 1 if layer_case1 is not None else None)
    case2_layers.append(layer_case2 + 1 if layer_case2 is not None else None)
    case3_layers.append(layer_case3 + 1 if layer_case3 is not None else None)
    case4_layers.append([l + 1 for l in layers_case4])
    combined_layers.append(matched_layers_1based)
    combined_TOP_vals.append(top_val)
    combined_BOT_vals.append(bot_val)
    match_distances.append(min_dist)

0/7248
100/7248
200/7248
300/7248
400/7248
500/7248
600/7248
700/7248
800/7248
900/7248


In [ ]:
DF_IPF_B_ = DF_IPF_B.copy() # copy, in case it doesn't workout or we want to test some things.

In [ ]:
# Assign to DataFrame
DF_IPF_B_['case1_L']     = case1_layers
DF_IPF_B_['case2_L']     = case2_layers
DF_IPF_B_['case3_L']     = case3_layers
DF_IPF_B_['case4_L']     = case4_layers
DF_IPF_B_['L_match']  = combined_layers
DF_IPF_B_['TOP_L_match']    = combined_TOP_vals
DF_IPF_B_['BOT_L_match']    = combined_BOT_vals
DF_IPF_B_['match_distance']  = match_distances


In [ ]:
IPF_Mdl_.sort_values('R', inplace=True)

In [ ]:
IPF_Mdl_

In [ ]:
def remove_L_from_combined_str(row):
    try:
        layer_list = row['L_match'].copy()
        layer_list.remove(row['L'])
    except ValueError:
        layer_list = row['L_match']
    return ', '.join(str(x) for x in layer_list)

In [ ]:
# Checks
IPF_Mdl_['match?'] = IPF_Mdl_.apply(lambda row: row['L'] in row['L_match'], axis=1)
IPF_Mdl_['L_extra'] = IPF_Mdl_.apply(remove_L_from_combined_str, axis=1)
IPF_Mdl_['TOP_match?'] = IPF_Mdl_.apply(lambda R: True if R['TOP_L_match'] > R['filtertoplevel'] else False, axis=1)
IPF_Mdl_['BOT_match?'] = IPF_Mdl_.apply(lambda R: True if R['BOT_L_match'] < R['filterbottomlevel'] else False, axis=1)

In [ ]:
IPF_Mdl_['case4_L'] = IPF_Mdl_['case4_L'].apply(lambda x: ', '.join(str(i) for i in x))
IPF_Mdl_['L_match'] = IPF_Mdl_['L_match'].apply(lambda x: ', '.join(str(i) for i in x))

In [ ]:
IPF_Mdl_[['case4_L', 'L_match', 'L_extra']] = IPF_Mdl_[['case4_L', 'L_match', 'L_extra']].astype(str)

In [ ]:
IPF_Mdl_.sort_values('id')

In [ ]:
IPF_Mdl_.to_csv('check_OBS_L_values.csv', index=None)

# 6. Stats

## 6.0. KGE of all Vs incorrectly matched. 

# 2. Read and edit check file.

In [ ]:
DF_check = pd.read_csv(r'..\..\\explore\check_OBS_L\check_OBS_L_values.csv')

In [ ]:
DF_check

In [ ]:
DF_OBS_IPF_MF6.columns

In [ ]:
DF_check.columns

In [ ]:
DF_edit = DF_check[['x', 'y', 'L_match', 'id', 'code', 'filterno', 'surfacelevel', 'filtertoplevel', 'filterbottomlevel']].copy()
DF_edit['L_match'] = DF_edit['L_match'].astype(str).str.split(', ')
DF_edit = DF_edit.explode('L_match').reset_index(drop=True)

In [ ]:
DF_edit.rename(columns={'x': 'X',
                        'y': 'Y',
                        'L_match': 'L',
                        'id': 'Id'}, inplace=True)

In [ ]:
help(DF_edit.merge)

In [ ]:
DF_edit = DF_edit.merge(right=DF_OBS_IPF_MF6[['Id', 'path']], right_on='Id', left_on='Id')

In [ ]:
DF_edit.head()

In [ ]:
os.makedirs(f'../../../models/NBr/In/OBS/{MdlN}', exist_ok=True)

In [ ]:
imod.formats.ipf.write(f'../../../models/NBr/In/OBS/{MdlN}/ijkset_selectie_{MdlN}.ipf', DF_edit, indexcolumn=10, assoc_ext='txt', nodata=1e+20)